In [2]:
# librerias a usar
import ee
import geemap

# Activar los pasos de la autenticación con la cuenta de GEE.
ee.Authenticate()

# Inicializar la librería ee.
ee.Initialize()

In [3]:
Map = geemap.Map()

# **********  Datos de entrada para los modelos de clasificación  ***************

# Se selecciona la colección de imagenes a utilizar. En este caso se trabaja con la collección Sentinel 2-A (reflectancia en superficie).
imagecollection =  ee.ImageCollection('COPERNICUS/S2_SR')
# Se carga el vector con el área/s a clasificar (Desde la compu o asset de GEE)
area = ee.FeatureCollection('users/COMPLETAR')
# Se carga el vector con los datos de referencia a terreno (para entrenar y validar)
rdata = ee.FeatureCollection('users/COMPLETAR')

# Se llama el MNC estival del INTA para la campaña 2021 para enmascarar áreas no agrícolas
mnc_ver_2021 = ee.Image('users/mnolasco/IG-SANCOR/mnc-verano-2021')
# Se llama el LandCover de la ESA para enmascarar áreas no agrícolas fuera de argentina
clc = ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019").select('discrete_classification')

# Se carga la capa hybrid de google en el visor
Map.add_basemap('HYBRID')
# Se carga en el visor las áreas a clasificar.  
Map.addLayer(area,{},'Zonas a calsificar', False)
# Se centra el visor en un punto
Map.setCenter(-63, -32, 7)

# Se define la paleta correspondiente al MNC 2021, y se carga el mapa en el visor
palette = ['FFF302','2DD200','FF9700','FFFFFF','1E8D00','B6B6B6','00FFE8','0046FF','FF1B00','FFDC00','000000','B6A431','000000','000000','000000','000000','FFE294','000000','FFAA94']
Map.addLayer(mnc_ver_2021,{'min': 10, 'max': 28, 'palette': palette},'Mapa Nacional de Cultivos 2020-2021', False)

# FILTRADO DE IMAGENES DE LA COLECCIÓN S2-A
# Se setea la fecha de inicio
start_date = '2020-06-01'
# Se setea la fecha de fin
end_date = '2021-06-01'
# Se setea el umbral para la cobertura de nubes
cloud = 20
# Se especifican las bandas de interes de S2
bands = ('B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12')

# Se filtra la colección S2 por bandas, fechas, cobertura de nubes y area de estudio.
S2_f = imagecollection.select(bands).filterDate(start_date, end_date).filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', cloud).filterBounds(area)

Map



Map(center=[-32, -63], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [ ]:
# Se puede pedir imprimir la propiedad de alguna imagen de la colección
image = ee.Image(S2_f.toList(S2_f.size()).get(0))
print(image.get('system:id').getInfo())


In [4]:
# Función para cambiar el nombre a las bandas de las imagenes
def changeBandNameS2(image):
    return image.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8','B8A','B11','B12'],
  ['BLUE', 'GREEN', 'RED', 'RED_EDGE1','RED_EDGE2','RED_EDGE3','NIR', 'RED_EDGE4', 'SWIR1','SWIR2'])
# Se aplica la función para cambiar el nombre a las bandas
S2n = changeBandNameS2(S2_f)
    
# Se define función para agregar la banda NDVI a cada imagen de la colección
def addNDVI(image):
    NDVI = image.normalizedDifference(['NIR', 'RED']).rename('NDVI')
    newImage = image.addBands(NDVI)
    return (newImage)

# Se puede aplicar la función para agregar la banda NDVI (OPCIONAL)
S2 = S2n#.map(addNDVI)

# Se crean mosaicos usando la función mediana segun los periodos de tiempo
median_image = S2_f.median()
MM_S2_a = S2.filter(ee.Filter.calendarRange(11,12, "month")).median()
MM_S2_b = S2.filter(ee.Filter.calendarRange(1,2, "month")).median()
MM_S2_c = S2.filter(ee.Filter.calendarRange(3,4, "month")).median()
#S2_d = S2.filter(ee.Filter.calendarRange(9,9, "month")).median()
#S2_e = S2.filter(ee.Filter.calendarRange(10,10, "month")).median()
#S2_f = S2.filter(ee.Filter.calendarRange(11,11, "month")).median()
#S2_g = S2.filter(ee.Filter.calendarRange(12,12, "month")).median()

# Se crean mosaicos usando la función quality mosaic segun los periodos de tiempo (Es necesario agregar la banda de NDVI)
#QM_S2_a = S2.filter(ee.Filter.calendarRange(9,10, "month")).qualityMosaic('NDVI')
#QM_S2_b = S2.filter(ee.Filter.calendarRange(11,12, "month")).qualityMosaic('NDVI')
#QM_S2_c = S2.filter(ee.Filter.calendarRange(1,2, "month")).qualityMosaic('NDVI')
#QM_S2_d = S2.filter(ee.Filter.calendarRange(3,5, "month")).qualityMosaic('NDVI')
#QM_S2_e = S2.filter(ee.Filter.calendarRange(1,1, "month")).qualityMosaic('NDVI').clip(area)
#QM_S2_f = S2.filter(ee.Filter.calendarRange(2,2, "month")).qualityMosaic('NDVI').clip(area)
#QM_S2_g = S2.filter(ee.Filter.calendarRange(3,3, "month")).qualityMosaic('NDVI').clip(area)
#QM_S2_h = S2.filter(ee.Filter.calendarRange(4,4, "month")).qualityMosaic('NDVI').clip(area)
#QM_S2_i = S2.filter(ee.Filter.calendarRange(5,5, "month")).qualityMosaic('NDVI').clip(area)


# Se seleccionan los mosaicos con los cuales trabajar
# El primero sirve de base para definir el area de trabajo
oneband = median_image.select(['B2']).rename(['BLUE0'])
S2_col = ee.ImageCollection([oneband, MM_S2_a, MM_S2_b, MM_S2_c]) 
#S2_col = ee.ImageCollection([oneband, QM_S2_a, QM_S2_b, QM_S2_c, QM_S2_d])

# Se define una función para apilar los mosaicos seleccionados y formar una imagen
def stackCollection(collection):
  # Create an initial image.
  first = ee.Image(collection.first()).select([])

  # Write a function that appends a band to an image.
  def appendBands(image, previous):
    return ee.Image(previous).addBands(image)
  
  return ee.Image(collection.iterate(appendBands, first))

# Se aplica la función de apilado de bandas
stacked = stackCollection(S2_col)

In [5]:
# Se setean las bandas a utilizar en la clasificación:
bands =['BLUE', 'GREEN', 'RED', 'RED_EDGE1', 'RED_EDGE2', 'RED_EDGE3', 'NIR', 'RED_EDGE4', 'SWIR1', 'SWIR2', 'BLUE_1', 'GREEN_1', 'RED_1', 'RED_EDGE1_1', 'RED_EDGE2_1', 'RED_EDGE3_1', 'NIR_1', 'RED_EDGE4_1', 'SWIR1_1', 'SWIR2_1', 'BLUE_2', 'GREEN_2', 'RED_2', 'RED_EDGE1_2', 'RED_EDGE2_2', 'RED_EDGE3_2', 'NIR_2', 'RED_EDGE4_2', 'SWIR1_2', 'SWIR2_2']

In [6]:
# LA SIGUIENTE TAREA PUEDE CONSUMIR BASTANTE TIEMPO, SE PUEDE EVITAR CON EL CUADRO ANTERIOR
# Función para generar lista con los nombres de bandas a utilizar (en este caso solo bandas espectrales - sin NDVI ni BLUE0)
def bnames(stack):
    bands = []
    img = stack.getInfo()['bands']
    l = len(img)
    for x in range(l):
        band = img[x]['id']
        if 'NDVI' not in band and band != 'BLUE0':
            bands.append(band)
    return bands
# Se aplica la función a las bandas apiladas.
bands = bnames(stacked)

In [6]:
# Se verifica la lista de bandas
print(bands)

['BLUE', 'GREEN', 'RED', 'RED_EDGE1', 'RED_EDGE2', 'RED_EDGE3', 'NIR', 'RED_EDGE4', 'SWIR1', 'SWIR2', 'BLUE_1', 'GREEN_1', 'RED_1', 'RED_EDGE1_1', 'RED_EDGE2_1', 'RED_EDGE3_1', 'NIR_1', 'RED_EDGE4_1', 'SWIR1_1', 'SWIR2_1', 'BLUE_2', 'GREEN_2', 'RED_2', 'RED_EDGE1_2', 'RED_EDGE2_2', 'RED_EDGE3_2', 'NIR_2', 'RED_EDGE4_2', 'SWIR1_2', 'SWIR2_2']


In [7]:
# Se selecciona la subarea de trabajo filtrando el vector

region = 'I' 
zona = area.filter(ee.Filter.eq('Zona', region)) # especificar atributo, en este caso "Zona"

# Cargar la región de trabajo en el visor
Map.addLayer(zona, {}, 'Zona a procesar', False)

In [47]:
# El mosaico de la zona especifica es guardado en un Asset de GEE.
# Esta tarea puede tardar varias horas dependiendo de las dimensiones del área a clasificar (si el área es chica no sería necesario crear el asset).
# Hacer correr la lista de tareas con anticipación.

task = ee.batch.Export.image.toAsset(**{
    'image': stacked.select(bands),
    'description': '2020-2021 summer median composite',
    'assetId': 'users/COMPLETAR/'+region+'_COMLETAR',
    'scale': 30,
    'region':((vz_PAS.geometry()).bounds()).getInfo()['coordinates'],
    'maxPixels': 1e10
})
task.start()

In [37]:
# Visualizar el mosaico generado (OPCIONAL)
image = ee.Image('users/COMPLETAR'+region+'COMPLETAR')
VisPar =  {'max': 2000, 'min': 0.0, 'gamma': 0.75, 'bands': ['RED_2', 'GREEN_2', 'BLUE_2']}
Map.addLayer (image, VisPar, 'Mosaico Color Real', False)
Map.centerObject(vz_PAS,7)


## Partición de los datos de referencia a terreno (entrenamiento y validación)

In [19]:

# Se carga los datos de referencia a terreno en el visor
Map.addLayer(rdata, {}, 'field reference data', False)

# Se selecionan los lotes dentro de la región especificada
frdata = rdata.filterBounds(zona.geometry())

# Se imprime el tamaño del conjunto de lotes de entrenamiento para verificar consistencia (OPCIONAL)
print(frdata.size().getInfo(), 'size field reference data')

# Se definen las clases de interes para la clasificación (en este caso agrícolas)
# Las clases se seleccionan/filtran por medio del atributo 'id'
SOJA = frdata.filter(ee.Filter.eq('id',1))
MAIZ = frdata.filter(ee.Filter.eq('id',3))
MAIZ_2 = frdata.filter(ee.Filter.eq('id',32))
GIRASOL = frdata.filter(ee.Filter.eq('id',4))
SORGO = frdata.filter(ee.Filter.eq('id',9))
ALGODON = frdata.filter(ee.Filter.eq('id',10))
ARROZ = frdata.filter(ee.Filter.eq('id',13))

# Se crea una FeatureCollection con las clases de interes seleccionadas segun la región    **************   COMPLETAR *************
vectors = SOJA.merge(MAIZ).merge(SORGO).merge(ARROZ)

# Se agrega una columna con valores aleatorios a cada lote. Se puede imprimir el primero (OPCIONAL)
vectors = vectors.randomColumn()
print(vectors.first().getInfo()['properties']['random'], 'random value in one reference data sample')

# El conjunto de datos de referencia es dividido en entrenamiento (70%) y validación (30%)
umbral = 0.7
trainingP = vectors.filter(ee.Filter.lt('random', umbral))
validationP = vectors.filter(ee.Filter.gt('random', umbral))
print(trainingP.size().getInfo(), 'frdata training partition size')
print(validationP.size().getInfo(), 'frdata validation partition size')

# La FeatureCollection es rasterizada
rdata_to_img = trainingP.reduceToImage(**{
    'properties': ['id'],
    'reducer': ee.Reducer.first()
}).rename('landcover');
print(rdata_to_img.getInfo(), 'imagen rasterizada properties')

# Se define la paleta de colores y valores para ver la imagen en el visor
palette = ['green', 'yellow','red', 'blue'] #******************   COMPLETAR *************
values = ee.List([1,3,9,13])   #  id de las clases clasificar        *****************   COMPLETAR *************
l = len(values.getInfo())
sequence = list(range(1, l+1, 1)) # Secuencia de números enteros separados por una unidad, a fin de facilitar la visualización
rdata_img = rdata_to_img.remap(values, sequence).rename('landcover')
Map.addLayer(rdata_img, { min:1, max: l,
  'palette': palette
}, 'frdata rasterized');

# Se muestrean 4000 puntos aleatorios por clase para ingresar al modelo de clasificación
trainSample = rdata_to_img.toByte().stratifiedSample(**{
  'numPoints': 4000,
  'classBand': 'landcover',
  'region': zona,
  'scale': 100,
  'geometries':True
});

2643 size field reference data
0.3715947801112811 random value in one reference data sample
941 frdata training partition size
406 frdata validation partition size
{'type': 'Image', 'bands': [{'id': 'landcover', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]} imagen rasterizada properties


In [57]:
# Se puede consultar la info del primer elemento del conjunto de validación (OPCIONAL)
print(testSample.first().getInfo())

{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [-61.8440665775664, -30.186537649910342]}, 'id': '0', 'properties': {'landcover': 1}}


In [111]:
# Imprimir cantidad de lotes a usar (entrenamiento + validación)
print(vectors.size().getInfo())

2641


## Clasificación

In [20]:
# Se entrena el algoritmo RF y se corre la clasificación con el mosaico guardado en el Asset *** ESPECIFICAR MASCARA AGRÍCOLA***
label = 'landcover'
image = ee.Image('users/COMPLETAR/'+region+'COMPLETAR')

# Con el conjunto de entrenamiento se realiza el muestreo sobre la imagen a clasificar
training = image.select(bands).sampleRegions(**{'collection': trainSample,'properties': [label],'scale': 20})

# Se especifican los parametros del clasificador Random Forest, y se realiza el entrenamiento del mismo.
trained = ee.Classifier.smileRandomForest(100).train(training, label,bands)

# Se realiza la clasificación de la imagen utilizando las mismas bandas que en el entrenamiento.
classified = image.select(bands).classify(trained).clip(zona)
                                                    
# Se realiza una reclasificación de los resultados para facilitar la visualización
mapa = classified.remap(values, sequence)
# Se enmascara la clase no agricola (incluido caña azucarera) usando el MNC 2021, o el CLC   **** ESPECIFICAR ****
mask = (mnc_ver_2021.remap([10,11,12,13,14,15,16,17,18,19,21,22,26,28],[1,1,1,1,2,1,1,1,1,1,2,2,1,1])).eq(1)
mask = clc.eq(40)
maskedComposite = mapa.updateMask(mask)
# Se carga el mapa resultante en el visor
Map.addLayer(maskedComposite,{'min':1, 'max':l,'palette': palette},'Mapa cultivos verano',False)
Map.centerObject(maskedComposite, 7)


#***************    la imagen se carga en el mapa anterior     ************

In [205]:
# Se puede ver el mapa sin enmascarar, en el visor (OPCIONAL)
Map.addLayer(mapa,{'min':1, 'max':l,'palette': palette},'Mapa crudo',False)

In [21]:
#**************************************************************************
# Pos procesamiento para reemplazar los pixeles aislados con valores de los pixeles alrededor
#**************************************************************************

# Se identifican los parches menores a 150 pixeles 
patchsize = maskedComposite.connectedPixelCount(150, False)

# Se corre un filtro de mayoría
filtered = maskedComposite.focal_mode(**{
    'radius': 100,
    'kernelType': 'square',
    'units': 'meters',
})

# las areas ocupadas por los parches menores a 101 pixeles son reemplazadas por el resultado del filtro de mayoria
connectedClassified =  maskedComposite.where(patchsize.lt(101),filtered)
# Se visualiza el mapa procesado en el visor
Map.addLayer(connectedClassified, {'min': 1, 'max': l, 'palette': palette}, 'summer map processed using Connected Pixels', True)

## Evaluación de la precisión

In [210]:
# Con el conjunto de validadción se realiza el muestreo sobre las bandas de la imagen que se clasificó.
# Se clasifica el conjunto de validación (OPCIONAL)
validation = image.select(bands).sampleRegions(**{
  'collection': validationP,
  'properties': ['id'],
  'scale': 100,
  'tileScale': 16
})

validated = validation.classify(trained)

In [211]:
# Se calcula la matriz de confusión y estadisticos PG y Kappa (OPCIONAL)
test_accuracy = validated.errorMatrix('id', 'classification')
print('PG:',test_accuracy.accuracy().getInfo()*100)
print('Kappa:', test_accuracy.kappa().getInfo())

PG: 67.16716716716716
Kappa: 0.35322703532783545


In [ ]:
# Exportar matriz de confusión a carpeta local de "Descargas". (OPCIONAL)
import csv
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
testing_csv = os.path.join(out_dir, 'test_accuracy COMPLETAR.csv')

   
with open(testing_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(test_accuracy.getInfo())

## Exportar datos raster

In [22]:
# se remapean los valores de cada pixel para exportar la imagen con los id de las clases
map_to_exp = connectedClassified.remap(sequence, values)

# Se exporta el mapa final a una carpeta de Google Drive
task = ee.batch.Export.image.toDrive(**{
    'image': map_to_exp,
    'description': region+'_COMPLETAR',
    'folder':'COMPLETAR CARPETA',
    'scale': 30,
    'region':((zona.geometry()).bounds()).getInfo()['coordinates'],
    'maxPixels': 1e10
})
task.start()

## Export datos vectoriales

In [23]:
# Exportar el conjunto de validación a una carpeta de Google Drive en formato KML.
task2 = ee.batch.Export.table.toDrive(**{
    'collection': validationP,
    'description': region+'COMPLETAR',
    'folder':'COMPLETAR',
'fileFormat': 'KML'
})
task2.start()